In [4]:
import pandas as pd
import json
import os
import random
from openai import OpenAI #estamos la clase concreta OpenAI del módulo openai
from dotenv import load_dotenv #importamos una función concreta del módulo
load_dotenv("template.env")

True

In [5]:
# Acceder a la clave de API de OpenAI
api_key = os.getenv("OPENAI_API_KEY")

# Asegurarte de que la clave de API se haya cargado correctamente
if api_key is None:
	raise ValueError("La clave de API no está configurada en el archivo .env")

client = OpenAI() #creando un objeto de la clase"

In [6]:
dataset_folder = os.getenv("DATASET_FOLDER")
dataset_path = str(dataset_folder) + "Alonso_2014_SpanishAoA.xlsx"

df = pd.read_excel(dataset_path)
#df = df.sample(5)
df.head()

c:\Users\Eneko\Documents\UniUPM\BECA\Lenguaje_interpret\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,word,averageAoA,SD,Min,Max,ItemZScore,OralFreq_Log,WrittenFreq_Subtlex-ESP_Log,WrittenFreq_LEXESP_Log,WrittenFreq_espal_Log,espal_max_lem_cat,max_lem_code,espal_es_num_syll
0,a,2.28,1.443352,1,6,-1.650147,4.851582,5.984858,4.960556,4.358684,ADPOSITION,SPS00,1.0
1,abajo,2.96,1.369642,1,6,-1.356683,2.615950,3.940915,2.750508,1.811229,ADVERB,RG,3.0
2,abandonado,6.06,1.658743,2,10,-0.584383,1.623249,2.872156,2.195900,1.374186,ADJECTIVE,AQ0MSP,5.0
3,abandonar,7.58,1.654801,4,11,-0.019940,1.806180,2.987666,2.434569,1.653307,VERB,VMN0000,4.0
4,abandono,7.22,1.940860,3,11,0.040498,1.602060,2.336460,2.167317,1.328129,NOUN,NCMS000,4.0


In [ ]:
#FUNCTION DECLARATION

def generate_task(index,prompt,desc):
	task = {
		"custom_id": f"task-{index}",
		"method": "POST",
		"url": "/v1/chat/completions",
		"body": {
			# Esto es lo que tendrías en tu llamada a la API de Chat Completions
			"model": "gpt-4o-mini",
			"temperature": 0,
			"response_format": { 
				"type": "json_object"
			},
			"messages": [
				{
				"role": "system",
				"content": prompt
				},
				{
				"role": "user",
				"content": desc
				}
			],
		}
	}
	return task

def generate_fine_tuning(prompt,word,aoa):
	f_t_line = {
		"messages": [
			{ "role": "system", "content": str(prompt) },
			{ "role": "user", "content": str(word) },
			{ "role": "assistant", "content": str(aoa) }
		]}
	return f_t_line

def create_file_from_tasks(tasks,file_name):
	with open(file_name, 'w') as file:
		for obj in tasks:
			file.write(json.dumps(obj) + '\n')


def create_fine_tunning_from_json(json_object,prompt):
	word = json_object["word"]
	aoa = json_object["averageAoA"]
	f_t_line = generate_fine_tuning(
		prompt,word,aoa
		)
	return f_t_line


def create_f_t_array_from_dataframe(df,prompt):
	tasks = []
	for index, row in df.iterrows():
		task = create_fine_tunning_from_json(row,prompt)
		tasks.append(task)
	return tasks


def get_line_file(file_name,line,extract_func):
	with open(file_name, 'r') as f:
		for line_number, theline in enumerate(f):
			if line_number == line:
				res = theline
				break
	res = json.loads(res)
	return extract_func(res)


def extract_input(new_line):
	return (new_line["messages"])


def upload_file(file_name: str, purpose: str) -> str:
    with open(file_name, "rb") as file_fd:
        response = client.files.create(file=file_fd, purpose=purpose)
    return response.id

In [8]:
#PROMPTS

#AGE PROMPT
categorize_system_prompt_paraphrase ='''
La edad de adquisición (AoA) de una palabra se refiere a la edad en la que se aprendió una palabra por primera vez. 
En concreto, cuándo una persona habría entendido por primera vez esa palabra si alguien la hubiera utilizado delante de ella, incluso cuando aún no la hubiera dicho, leído o escrito. 
Calcule la edad media de adquisición (AoA) de la palabra {palabra} para un hablante nativo de español.

El formato de salida debe ser un objeto JSON: {AoA: número //AoA de la palabra expresado en años, puede incluir decimales, Word: palabra //string}
'''

In [9]:
#SET output folder
output_folder = os.getenv("OUTPUT_FOLDER")
def out_file(file_name): return (str(output_folder) + file_name)

In [15]:
#CREATE tasks from the database
f_t_array = [create_f_t_array_from_dataframe(df,categorize_system_prompt_paraphrase)]
f_t_file_array = [out_file("batch_job_mmlu_f_t_aoa_alonso.jsonl"),out_file("batch_job_mmlu_check_aoa_alonso.jsonl")]

In [20]:
#SEPARATE words in piles to fine-tune or check
num_task_train = 2000

#check all tasks in task array
max_num_task = 0
for i in range(0,len(f_t_array)):
	max_num_task += len(f_t_array[i])

#generate random task indexes
random_num_array = []
for i in range(0,num_task_train):
	new_num = random.randrange(0,max_num_task)
	while(new_num in random_num_array):
		new_num = random.randrange(0,max_num_task)
	random_num_array.append(new_num)

#create training array and check array
indiv = [[] for Null in range(len(f_t_file_array))]

index_com = 0
for i in range(0,len(f_t_array)):
	tasks = f_t_array[i]
	for j in range(0,len(tasks)):
		if ((index_com+j)in random_num_array):
			indiv[0].append(tasks[j])
		else:
			indiv[1].append(tasks[j])
	index_com += len(f_t_array[i])

#create files
for i in range(0,len(indiv)):
	create_file_from_tasks(indiv[i],f_t_file_array[i])

In [18]:
#TRAIN fine tuning file
training_file_id = upload_file(f_t_file_array[0], "fine-tune")

job = client.fine_tuning.jobs.create(
		training_file = training_file_id,
		model = "gpt-4o-mini-2024-07-18",
	)

BadRequestError: Error code: 400 - {'error': {'message': 'Invalid file format for Fine-Tuning API. Must be .jsonl', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
#CHECK fine tuning progress
f_t_job = client.fine_tuning.jobs.retrieve(job.id)
print(f_t_job.status)
print(f_t_job.id)

In [ ]:
#EXTRACT fine tuned model
fine_tuned_model_id = f_t_job.fine_tuned_model
print(f_t_job)
print(fine_tuned_model_id)

In [ ]:
#TEST the rest of the words in database
test_file = f_t_file_array[1]
rows = []

with open(test_file, 'r') as f:
	lines = len(f.readlines())

for i in range(0,lines):
	test_line = get_line_file(test_file,i,extract_input)

	test_messages = []
	test_messages.append(test_line[0])
	test_messages.append(test_line[1])
	response = client.chat.completions.create(
		model=fine_tuned_model_id, messages=test_messages, temperature=0
	)
	rows.append({
		"palabra":test_line[1]["content"],
		"AoA_ini":test_line[2]["content"],
		"AoA_IA":response.choices[0].message.content
	})

In [ ]:
#SAVE ouput in a .xlsx file
file_name = out_file("Prov_f_t_Results_"+str(num_task_train)+".xlsx")
clean_dtset = pd.DataFrame(rows)

#CALCULATE correlation coeff
cmp_clm_1 = "AoA_ini"
cmp_clm_2 = "AoA_IA"

pearson_corr = clean_dtset[[cmp_clm_1, cmp_clm_2]].corr('pearson')
pearson_corr = pearson_corr[cmp_clm_1][cmp_clm_2]
print(f"Pearson_Corr: {pearson_corr}")

spearman_corr = clean_dtset[[cmp_clm_1, cmp_clm_2]].corr('spearman')
spearman_corr = spearman_corr[cmp_clm_1][cmp_clm_2]
print(f"Spearman_Corr: {spearman_corr}")

stats = pd.DataFrame([{
	"Pearson_Corr":pearson_corr,
	"Spearman_Corr":spearman_corr
	}])

#WRITE file
with pd.ExcelWriter(file_name) as writer:
	clean_dtset.to_excel(writer, sheet_name='Results',index=False)
	stats.to_excel(writer, sheet_name='Stats',index=False)

#SAVE ouput in a .xlsx file
file_name = out_file("Prov_f_t_Results.xlsx")
clean_dtset = pd.DataFrame(rows)

with pd.ExcelWriter(file_name) as writer:
	clean_dtset.to_excel(writer, sheet_name='Results',index=False)

In [ ]:
dataset_path = out_file("Prov_f_t_Results_300.xlsx")
df = pd.read_excel(dataset_path)

cmp_clm_1 = "AoA_ini"
cmp_clm_2 = "AoA_IA"

pearson_corr = df[[cmp_clm_1, cmp_clm_2]].corr('pearson')
pearson_corr = pearson_corr[cmp_clm_1][cmp_clm_2]
print(f"Pearson_Corr: {pearson_corr}")

spearman_corr = df[[cmp_clm_1, cmp_clm_2]].corr('spearman')
spearman_corr = spearman_corr[cmp_clm_1][cmp_clm_2]
print(f"Spearman_Corr: {spearman_corr}")

In [ ]:
word = "desconfinar"
test_messages_oneWord = [{"role": "system", "content": "\nLa edad de adquisici\\u00f3n (AoA) de una palabra se refiere a la edad en la que se aprendi\\u00f3 una palabra por primera vez. \\nEn concreto, cu\\u00e1ndo una persona habr\\u00eda entendido por primera vez esa palabra si alguien la hubiera utilizado delante de ella, incluso cuando a\\u00fan no la hubiera dicho, le\\u00eddo o escrito. \\nCalcule la edad media de adquisici\\u00f3n (AoA) de la palabra {palabra} para un hablante nativo de espa\\u00f1ol.\\n\\nEl formato de salida debe ser un objeto JSON: {AoA: n\\u00famero //AoA de la palabra expresado en a\\u00f1os, puede incluir decimales, Word: palabra //string}\n"}, {"role": "user", "content": word}]
response = client.chat.completions.create(
	model=fine_tuned_model_id, messages=test_messages, temperature=0
	)
print(response.choices[0].message.content)

line = {"messages": [
	{"role": "system", "content": "\nLa edad de adquisici\\u00f3n (AoA) de una palabra se refiere a la edad en la que se aprendi\\u00f3 una palabra por primera vez. \\nEn concreto, cu\\u00e1ndo una persona habr\\u00eda entendido por primera vez esa palabra si alguien la hubiera utilizado delante de ella, incluso cuando a\\u00fan no la hubiera dicho, le\\u00eddo o escrito. \\nCalcule la edad media de adquisici\\u00f3n (AoA) de la palabra {palabra} para un hablante nativo de espa\\u00f1ol.\\n\\nEl formato de salida debe ser un objeto JSON: {AoA: n\\u00famero //AoA de la palabra expresado en a\\u00f1os, puede incluir decimales, Word: palabra //string}\n"},
	{"role": "user", "content": word}, 
	{"role": "assistant", "content": response.choices[0].message.content}
	]}

with open(out_file("indiv_words_analisis.txt"), 'a') as file1:
	file1.write(str(line))

In [ ]:
#CALCULATE correlation coeff
cmp_clm_1 = "AoA_ini"
cmp_clm_2 = "AoA_IA"

pearson_corr = clean_dtset[[cmp_clm_1, cmp_clm_2]].corr('pearson')
print(f"Pearson_Corr: {pearson_corr[cmp_clm_1][cmp_clm_2]}")

spearman_corr = clean_dtset[[cmp_clm_1, cmp_clm_2]].corr('spearman')
print(f"Spearman_Corr: {spearman_corr[cmp_clm_1][cmp_clm_2]}")